Import libraries

In [1]:
import time
from urllib.error import HTTPError
from urllib.error import URLError

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import netCDF4

import PyCO2SYS as pyco2

from erddapy import ERDDAP
from erddapy.doc_helpers import show_iframe
from erddapy import servers

import cartopy
cartopy.config['data_dir'] = './maps'
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as feat

Retrieve list of erddap servers that have carbon variables, using the CF standard names. Quicker using erddap.com

In [ ]:
stdnames=['surface_partial_pressure_of_carbon_dioxide_in_sea_water',
    'fugacity_of_carbon_dioxide_in_sea_water',
    'mole_concentration_of_dissolved_inorganic_carbon_in_sea_water',
    'moles_of_dissolved_inorganic_carbon_per_unit_mass_in_sea_water',
    'sea_water_ph_reported_on_total_scale',
    'sea_water_alkalinity_expressed_as_mole_equivalent']

starttime=time.time()

for s in servers:
    e=ERDDAP(s)
    for v in stdnames:
        kw= {"standard_name": v}
        search_url = e.get_search_url(response="csv", **kw)
        try:
            search=pd.read_csv(search_url)
        except HTTPError:
            pass
            #print("No", stdname, "data in the server", s)
        except URLError:
            pass
            #print(s,"throws an URL error")
        else:
            print(s,"has", {len(set(search["tabledap"].dropna()))},"tabledap datasets with",v)
            print(search[["Dataset ID","Title"]])
            if len(set(search["griddap"].dropna())) > 0:
                print(s,"has", {len(set(search["griddap"].dropna()))},"tabledap datasets with",v)

            

endtime=time.time()-starttime
print("\n Search took", endtime/60,"minutes")


Create a dictionary in order to rename all columns to the same

In [2]:
vardict ={'id':'ID', 'doi':'Source_DOI','datevec':'DATEVECTOR','unixd':'UNIXDATE',
          'lat':'LATITUDE','lon':'LONGITUDE','dep':'DEPTH', 'pres':'PRESSURE',
          'temp':'TEMPERATURE', 'tempf':'TEMPERATURE_FLAG',
          'sal':'SALINITY','salf':'SALINITY_FLAG',
          'dic' : 'DIC', 'dicf':'DIC_FLAG', 'dicc':'DIC_CALCULATION',
          'alk': 'ALKALINITY', 'alkf':'ALKALINTY_FLAG', 'alkc':'ALKALINITY_CALCULATION',
          'ph': 'pH_TS','phf': 'pH_FLAG','phc': 'pH_CALCULATION',
          'fco2w':'FCO2_W','fco2wf':'FCO2W_FLAG','fco2wc': 'FCO2_CALCULATION','fco2wac':'ACCURACY_FCO2',}

Get list of datasets that contain pH. Constrain by a particular time frame to avoid downloading a too large dataset

In [19]:
i = ERDDAP(server="EMODNET", protocol='tabledap')
search_url = i.get_search_url(response="csv", **{
    "standard_name": "sea_water_ph_reported_on_total_scale",
    "min_time": "2019-03-01",
    "max_time": "2019-03-15"})
search = pd.read_csv(search_url)
print(search[["Title","Dataset ID"]].values)

[['EMODnet Physics - Collection of  (PHPH) TimeSeries  Min/Max/Mean -  MultiPointTimeSeriesObservation'
  'EP_ERD_INT_PHPH_AL_TS_MINMAXMEAN']
 ['EMODnet Physics - Collection of Ph (PHPH) Profiles -  MultiPointProfileObservation'
  'EP_ERD_INT_PHPH_AL_PR_NRT']
 ['EMODnet Physics - Collection of Ph (PHPH) TimeSeries -  MultiPointTimeSeriesObservation'
  'EP_ERD_INT_PHPH_AL_TS_NRT']]


Get information about the dataset EP_ERD_INT_PHPH_AL_PR_NRT: variable names, time coverage, pH attributes

In [21]:
info = pd.read_csv(i.get_info_url(dataset_id="EP_ERD_INT_PHPH_AL_PR_NRT", response="csv"))
# List dataset variables
print(", ".join(info.loc[info["Row Type"] == "variable", "Variable Name"]))
print("\n")
# Time coverage
print(info.loc[info["Attribute Name"] == "time_coverage_start", "Value"])
print(info.loc[info["Attribute Name"] == "time_coverage_end", "Value"])
print("\n")
# pH attributes
print(info.loc[info["Variable Name"] == "PHPH", :])
print("\n")
print(info.loc[info["Variable Name"] == "PHPH_QC", :])

EP_PLATFORM_ID, EP_PLATFORM_TYPE, EP_PLATFORM_CODE, EP_PLATFORM_LINK, time, TIME_QC, depth, DEPTH_QC, pres, PRES_QC, latitude, longitude, POSITION_QC, DIRECTION, PHPH, PHPH_QC, PHPH_DM, site_code, platform_code, platform_name, pi_name, area, author, source, contributor_name, contributor_url, data_assembly_center, institution_edmo_code, institution_references, institution, wmo_platform_code


61    1921-09-12T19:00:00Z
Name: Value, dtype: object
60    2021-10-09T23:57:14Z
Name: Value, dtype: object


      Row Type Variable Name        Attribute Name Data Type  \
166   variable          PHPH                   NaN     float   
167  attribute          PHPH            _FillValue     float   
168  attribute          PHPH          actual_range     float   
169  attribute          PHPH    ep_parameter_group    String   
170  attribute          PHPH             long_name    String   
171  attribute          PHPH         missing_value     float   
172  attribute          PHPH                   

Get the data, filtered by only good values of pH. 

In [30]:
i.dataset_id="EP_ERD_INT_PHPH_AL_PR_NRT"
i.response="csv"

# List variables to retrieve
# Axes variables
varaxisname=i.get_var_by_attr(axis=lambda v: v in["X","Y","Z","T"])
varsnames=['PHPH']
varsqcnames=i.get_var_by_attr(long_name="quality flag")
i.variables=varaxisname + varsnames + varsqcnames + ['PHPH_QC']

# Constraints
i.constraints= {
    "time>=": "2017-01-01T00:00:00Z",
    "time<=": "2019-12-31T23:59:59Z",
    "PHPH_QC=": 2,
    "PHPH_QC=": 3}

dtype=object
df_emodnet = i.to_pandas()

# Rename the columns to a common nomenclature
df_emodnet.rename(
    columns={'latitude (degrees_north)': vardict['lat'], 'longitude (degrees_east)': vardict['lon'],
             'pres (dbar)': vardict['pres'], 'time (UTC)': vardict['datevec'],
             'temp_adjusted (degree_Celsius)': vardict['temp'], 'temp_adjusted_qc':vardict['tempf'],
             'PHPH (1)': vardict['ph'], 'ph_in_situ_total_qc': vardict['phf']},
        inplace=True)

# Variables retrieved
print(df_emodnet.columns)
# Size of matrix
print(df_emodnet.shape)

Index(['depth (m)', 'LATITUDE', 'LONGITUDE', 'PRESSURE', 'DATEVECTOR', 'pH_TS',
       'POSITION_QC (1)', 'TIME_QC (1)', 'DEPTH_QC (1)', 'PRES_QC (1)',
       'PHPH_QC (1)'],
      dtype='object')
(635824, 11)


Similarly for IFREMER ERDDAP

In [32]:
i = ERDDAP(server="IFREMER", protocol='tabledap')
search_url = i.get_search_url(response="csv", **{
    "cf_standard_name": "sea_water_ph_reported_on_total_scale", # they use cf_standard
    "min_time": "2019-03-01",
    "max_time": "2019-03-15"})
search = pd.read_csv(search_url)
print(search[["Title","Dataset ID"]].values)

[['Argo Float Measurements' 'ArgoFloats']
 ['Argo float synthetic vertical profiles : BGC data'
  'ArgoFloats-synthetic-BGC']
 ['Global Ocean, In Situ Observation Copernicus (Copernicus Fishing Observing System)'
  'copernicus-fos']
 ['OceanGliders GDAC trajectories' 'OceanGlidersGDACTrajectories']
 ['SDC Baltic Sea Aggregation V2' 'SDC_BAL_AGG_V2']
 ['SDC Mediterranean Sea Aggregation V2' 'SDC_MED_AGG_V2']
 ['SDC North Atlantic Aggregation V2' 'SDC_NATL_AGG_V2']]


Pick the BGC-Argo synthetic profiles dataset

In [51]:
i.dataset_id="ArgoFloats-synthetic-BGC"
i.response="csv"

info = pd.read_csv(i.get_info_url())
# List dataset variables (very long!)
print(", ".join(info.loc[info["Row Type"] == "variable", "Variable Name"]))
print("\n")
# Time coverage
print(info.loc[info["Attribute Name"] == "time_coverage_start", "Value"])
print(info.loc[info["Attribute Name"] == "time_coverage_end", "Value"])
print("\n")
# pH attributes
print(i.get_var_by_attr(dataset_id="ArgoFloats-synthetic-BGC",cf_standard_name="sea_water_ph_reported_on_total_scale"))
print(info.loc[info["Variable Name"] == "ph_in_situ_total", :])
print("\n")
print(info.loc[info["Variable Name"] == "ph_in_situ_total_qc", :]) 
# QC is a string:

positioning_system, wmo_inst_type, platform_type, parameter_data_mode, data_centre, format_version, handbook_version, data_type, time, time_qc, time_location, latitude, longitude, position_qc, platform_number, cycle_number, direction, config_mission_number, cndc, cndc_qc, cndc_dpres, cndc_adjusted, cndc_adjusted_qc, cndc_adjusted_error, pres, pres_qc, pres_dpres, pres_adjusted, pres_adjusted_qc, pres_adjusted_error, psal, psal_qc, psal_dpres, psal_adjusted, psal_adjusted_qc, psal_adjusted_error, temp, temp_qc, temp_dpres, temp_adjusted, temp_adjusted_qc, temp_adjusted_error, doxy, doxy_qc, doxy_dpres, doxy_adjusted, doxy_adjusted_qc, doxy_adjusted_error, temp_doxy, temp_doxy_qc, temp_doxy_dpres, temp_doxy_adjusted, temp_doxy_adjusted_qc, temp_doxy_adjusted_error, temp_voltage_doxy, temp_voltage_doxy_qc, temp_voltage_doxy_dpres, temp_voltage_doxy_adjusted, temp_voltage_doxy_adjusted_qc, temp_voltage_doxy_adjusted_error, voltage_doxy, voltage_doxy_qc, voltage_doxy_dpres, voltage_doxy_adj

['ph_in_situ_total']
       Row Type     Variable Name     Attribute Name Data Type  \
1050   variable  ph_in_situ_total                NaN     float   
1051  attribute  ph_in_situ_total         _FillValue     float   
1052  attribute  ph_in_situ_total       actual_range     float   
1053  attribute  ph_in_situ_total   cf_standard_name    String   
1054  attribute  ph_in_situ_total          long_name    String   
1055  attribute  ph_in_situ_total  sdn_parameter_urn    String   
1056  attribute  ph_in_situ_total              units    String   
1057  attribute  ph_in_situ_total          valid_max     float   
1058  attribute  ph_in_situ_total          valid_min     float   

                                     Value  
1050                                   NaN  
1051                               99999.0  
1052                   -31.72798, 71.51507  
1053  sea_water_ph_reported_on_total_scale  
1054                                    pH  
1055                     SDN:P01::PHMASSXX  
105

The dataset has 540 variables in total. Pick only the ones we're interested in..

In [52]:
# Reduce number of variables; total columns is 540
# Axes variables
varaxisname=i.get_var_by_attr(axis=lambda v: v in["X","Y","Z","T"])
varsnames=['ph_in_situ_total']
varsqcnames=i.get_var_by_attr(long_name="quality flag")
i.variables=varaxisname + varsnames + varsqcnames + ['ph_in_situ_total_qc']

i.constraints= {
    "time>=": "2017-01-01T00:00:00Z",
    "time<=": "2019-12-31T23:59:59Z",
    "ph_in_situ_total_qc=": '1'} #  good data
        
dtype=object
df_ifremer = i.to_pandas()
print(df_ifremer.columns)
print(df_ifremer.shape)

HTTPError: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>500 Internal Server Error</title>
</head><body>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error or
misconfiguration and was unable to complete
your request.</p>
<p>Please contact the server administrator at 
 assistance@ifremer.fr to inform them of the time this error occurred,
 and the actions you performed just before this error.</p>
<p>More information about this error may be available
in the server error log.</p>
</body></html>


In [ ]:
df_ifremer.rename(
    columns={'latitude (degrees_north)': vardict['lat'], 'longitude (degrees_east)': vardict['lon'],
             'pres_adjusted (decibar)': vardict['pres'], 'time (UTC)': vardict['datevec'],
             'temp_adjusted (degree_Celsius)': vardict['temp'], 'temp_adjusted_qc':vardict['tempf'],
             'ph_in_situ_total_adjusted (dimensionless)': vardict['ph'], 'ph_in_situ_total_qc': vardict['phf']},
        inplace=True)
df_ifremer['SOURCE']='BGC-Argo (IFREMER)'

Map scatterplot

In [ ]:
dflist=[df_ifremer,df_emodnet]
df=pd.concat(dflist)

proj=ccrs.cartopy.crs.Miller()
plt.figure(dpi=200)
ax = plt.axes(projection=proj)
if ((df[vardict['lon']].min() < -175) & (df[vardict['lon']].max() > 175) & 
    (df[vardict['lat']].min() <-85) & (df[vardict['lat']].max() > 85)) :
    ax.set_extent([df[vardict['lon']].min()-5,df[vardict['lon']].max()+5, 
                   df[vardict['lat']].max()+5,df[vardict['lat']].min()-5])
ax.stock_img()
ax.coastlines()

sc=ax.scatter(df_ifremer[vardict['lon']],df_ifremer[vardict['lat']],
    c='r',s=5,label='BGC-Argo IFREMER',
    transform=ccrs.PlateCarree())
sc=ax.scatter(df_emodnet[vardict['lon']],df_emodnet[vardict['lat']],
    c='b',s=1,label='EMODNET',
    transform=ccrs.PlateCarree())
    
#plt.colorbar(sc)
ax.legend(loc='lower center')
plt.show()

Latitude-depth scatterplots

In [ ]:
# latitude-depth scatterplots. Atlantic

plt.figure(figsize=(20,6), dpi=200)
ax = plt.axes()
sc=ax.scatter(df_ifremer[vardict['lat']],
           df_ifremer[vardict['pres']],
           c=df_ifremer[vardict['ph']])
#,
#          s=5,label='BGC-Argo IFREMER')
ax.scatter(df_emodnet[vardict['lat']],
           df_emodnet[vardict['pres']],
           c=df_emodnet[vardict['ph']],
           s=1,label='EMODNET')
plt.xticks(rotation=45);
ax.invert_yaxis();
plt.colorbar(sc);
#ax.legend(loc='lower center')